In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout

# Load dataset
data = pd.read_csv('EURUSD_tick_OK-2.csv')

# Preprocess dataset (Adjust columns as per your dataset)
X = data.iloc[:, :-1].values  # Features
y = data.iloc[:, -1].values   # Target

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Reshape data for Conv1D
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Calculate the number of epochs
sid = "345678"  # Replace with your SID
Z = int(sid[-2])
Y = int(sid[-1])

if Z == 0 and Y == 0:
    epochs = 10
elif Z == 0:
    epochs = 10 + Y
else:
    epochs = Z + Y

# Define the CNN model
model = Sequential([
    Conv1D(filters=32, kernel_size=2, activation='relu', input_shape=(X_train.shape[1], 1)),  # kernel_size reduced to 2
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='linear')  # Output layer
])

# Compile the model
model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mae'])

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=50, verbose=1)

# Evaluate the model
mae = model.evaluate(X_test, y_test, verbose=0)[1]
print(f"Test MAE of the modified CNN: {mae:.4f}")

# Display the model architecture
model.summary()


Epoch 1/15


c:\Users\96659\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


17600/17600 ━━━━━━━━━━━━━━━━━━━━ 38s 2ms/step - loss: 0.1238 - mae: 0.1238 - val_loss: 0.1213 - val_mae: 0.1213
Epoch 2/15
17600/17600 ━━━━━━━━━━━━━━━━━━━━ 69s 4ms/step - loss: 0.1210 - mae: 0.1210 - val_loss: 0.1213 - val_mae: 0.1213
Epoch 3/15
17600/17600 ━━━━━━━━━━━━━━━━━━━━ 73s 4ms/step - loss: 0.1212 - mae: 0.1212 - val_loss: 0.1213 - val_mae: 0.1213
Epoch 4/15
17600/17600 ━━━━━━━━━━━━━━━━━━━━ 35s 2ms/step - loss: 0.1209 - mae: 0.1209 - val_loss: 0.1214 - val_mae: 0.1214
Epoch 5/15
17600/17600 ━━━━━━━━━━━━━━━━━━━━ 41s 2ms/step - loss: 0.1210 - mae: 0.1210 - val_loss: 0.1214 - val_mae: 0.1214
Epoch 6/15
17600/17600 ━━━━━━━━━━━━━━━━━━━━ 44s 2ms/step - loss: 0.1209 - mae: 0.1209 - val_loss: 0.1215 - val_mae: 0.1215
Epoch 7/15
17600/17600 ━━━━━━━━━━━━━━━━━━━━ 33s 2ms/step - loss: 0.1210 - mae: 0.1210 - val_loss: 0.1213 - val_mae: 0.1213
Epoch 8/15
17600/17600 ━━━━━━━━━━━━━━━━━━━━ 36s 2ms/step - loss: 0.1213 - mae: 0.1213 - val_loss: 0.1213 - val_mae: 0.1213
Epoch 9/15
17600/17600 ━━━━

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_1 (Conv1D)               │ (None, 3, 32)          │            96 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 1, 32)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,821 (26.65 KB)

 Trainable params: 2,273 (8.88 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 4,548 (17.77 KB)